# Data Reading and Pre Filter

Data Download from source (http://jmcauley.ucsd.edu/data/amazon/) and from google drive

Due to data size, the data have to be process in chunk and pre-filter based on 
helpfulness ratio

There are some command line shown in this section to assist with from/to 
google drive and google collab such as download, copy, zip, save

In [ ]:
# Download from source
# Need to download all data category but only shows one as an example here
!wget "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books.json.gz"

In [ ]:
# Copy from google collab to google drive
!cp -r "/content/drive/MyDrive/Test/reviews_Movies_and_TV.json.gz" "/content/drive/MyDrive/Test"

In [ ]:
# Copy from google drive to google collab
# Change path
!cp -r "/content/drive/MyDrive/Test/reviews_Electronics.json.gz" "/content/"

The json file is compressed in gz file therefore need uncompressed

In [ ]:
# Compress then delete the gz file
!gzip -dk "/content/reviews_Electronics.json.gz" | tar -xf -

swifter: help with speeding up data frame function such as .apply

In [ ]:
!pip install -q swifter

Please set the parameter below for functionality required

*   DATA_PROCESSING: Product category processing
*   JSON_PREPROSESSING: Read from JSON file processing
*   SAVE_JSON_PREPROSESSING: Save the JSON data into CSV



In [ ]:
# Category to process: CD, BOOK, ELEC, MOVIE
DATA_CATEGORY = 'ELEC'

# JSON Preprocessing: read from json and filter based on helpfulness ratio
JSON_PREPROSESSING = True
SAVE_JSON_PREPROSESSING = True

if DATA_CATEGORY == 'CD':

    DATA_NAME = 'CDs_and_Vinyl'

elif DATA_CATEGORY == 'BOOK':

    DATA_NAME = 'Books'

elif DATA_CATEGORY == 'ELEC':

    DATA_NAME = 'Electronics'

elif DATA_CATEGORY == 'MOVIE':

    DATA_NAME = 'Movies_and_TV'
    

In [ ]:
import pandas as pd
import swifter
import numpy as np
import datetime

In [ ]:
def get_ratio(a, b):
  if a > 0:
    return a/b
  else:
    return 0

def get_deviation_extreme(rate, min, max):
  abs_min = abs(rate-min)
  abs_max = abs(rate-max)

  if (abs_min >= abs_max):
    return abs_min
  elif (abs_min < abs_max):
    return abs_max

In [ ]:
# JSON read and preprocessing: 
# Doing it in chunk because of memory issue
# Prefilter with helpful ratio because of memory issue
def read_json_file(path,helpful_ratio,unhelpful_ratio):

  good_columns = [
   'reviewerID',
  'asin',
  'reviewerName',
  'helpful',
  'reviewText',
  'overall',
  'summary',
  'unixReviewTime',
  'reviewTime',
  ]

  df = pd.DataFrame(columns= good_columns)

  i = 0

  chunks = pd.read_json(path, lines=True, chunksize = 1000000)

  for chunk in chunks:

    i += len(chunk.index)

    chunk['total'] = chunk['helpful'].swifter.progress_bar(False).apply(lambda x: x[1])
    

    chunk['positive'] = chunk['helpful'].swifter.progress_bar(False).apply(lambda x: x[0])
    chunk['helpful_ratio'] = chunk['positive'] / chunk['total']
  
    df = df.append(chunk)

    if (i % 1000000) == 0:
      now = datetime.datetime.now()
      print("Number of rows processed:", i,now) 

  print("Total row", i)  

  return df

In [ ]:
if JSON_PREPROSESSING == True:

  df = read_json_file('/content/reviews_' + DATA_NAME + '.json', 0.75, 0.35)

In [ ]:
df.info()

In [ ]:
df['product_max_rating'] = df.groupby(['asin'])[['overall']].transform('max')
df['product_min_rating'] = df.groupby(['asin'])[['overall']].transform('min')
df['product_deviation_extreme_rate'] = df.swifter.apply(lambda x : \
                    get_deviation_extreme( x['overall'],x['product_max_rating'],\
                                          x['product_min_rating']),axis=1)
df['product_rating'] = df.groupby(['asin'])[['overall']].transform('mean')
df['product_std_rating'] = df.groupby(['asin'])[['overall']].transform('std')
df['product_total_review'] = df.groupby(['asin'])[['reviewerID']].transform('count')

df['reviewer_mean_rating'] = df.groupby(['reviewerID'])[['overall']].transform('mean')
df['reviewer_deviation_average_rate'] = abs(df['reviewer_mean_rating'] - df['overall'])
df['reviewer_total_helpfulness_vote'] = df.groupby(['reviewerID'])[['total']].transform('sum')
df['reviewer_total_review'] = df.groupby(['reviewerID'])[['reviewerID']].transform('count')
df['reviewer_std_rating'] = df.groupby(['reviewerID'])[['overall']].transform('std')

In [ ]:
df = df[df['total'] >= 10 ]

In [ ]:
df.info()

In [ ]:
df = df[(df['helpful_ratio'] > 0.75) | (df['helpful_ratio'] < 0.35)]

In [ ]:
df.info()

In [ ]:
df.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

df.describe()

In [ ]:
# Save the result from JSON preprocessing result
# as csv to google collab

df = df.reset_index(drop=True)

if SAVE_JSON_PREPROSESSING == True:

  path = '/content/' + DATA_NAME + '.csv'

  with open(path, 'w', encoding = 'utf-8-sig') as f:
    df.to_csv(f)

Save the processed data as zip into Google Drive

In [ ]:
#!zip -r '/content/drive/MyDrive/Test/Processed/CDs_and_Vinyl.zip' '/content/CDs_and_Vinyl.csv'

In [ ]:
#!zip -r '/content/drive/MyDrive/Test/Processed/Movies_and_TV.zip' '/content/Movies_and_TV.csv'

  adding: content/Movies_and_TV.csv (deflated 60%)


In [ ]:
#!zip -r '/content/drive/MyDrive/Test/Processed/Electronics.zip' '/content/Electronics.csv'

  adding: content/Electronics.csv (deflated 61%)


In [ ]:
!zip -r '/content/drive/MyDrive/Test/Processed/Books.zip' '/content/Books.csv'

  adding: content/Books.csv (deflated 60%)
